# 04 - AI Orchestration with Azure Cognitive Search

In this lab, we will do a deeper dive around the Azure Cognitive Search vector store and different ways to interact with it.

## Create Azure Cognitive Search Vector Store in Azure

First, we need to create an Azure Cognitive Search service in Azure, which will act as a vector store. We'll use the Azure CLI to do this.

**NOTE:** Update **`<INITIALS>`** to make the name unique.

In [ ]:
// Execute the following commands using the Azure CLI to create the Azure Cognitive Search resource in Azure.
RESOURCE_GROUP="azure-cognitive-search-rg"
LOCATION="westeurope"
NAME="acs-vectorstore-<INITIALS>"
az group create --name $RESOURCE_GROUP --location $LOCATION
az search service create -g $RESOURCE_GROUP -n $NAME -l $LOCATION --sku Basic --partition-count 1 --replica-count 1

Next, we need to find and update the following values in the `.env` file with the Azure Cognitive Search **endpoint**, **admin key**, and **index name** values. Use the Azure Portal or CLI.

```
AZURE_COGNITIVE_SEARCH_SERVICE_NAME = "<YOUR AZURE COGNITIVE SEARCH SERVICE NAME - e.g. cognitive-search-service>"
AZURE_COGNITIVE_SEARCH_ENDPOINT_NAME = "<YOUR AZURE COGNITIVE SEARCH ENDPOINT NAME - e.g. https://cognitive-search-service.search.windows.net"
AZURE_COGNITIVE_SEARCH_INDEX_NAME = "<YOUR AZURE COGNITIVE SEARCH INDEX NAME - e.g. cognitive-search-index>"
AZURE_COGNITIVE_SEARCH_API_KEY = "<YOUR AZURE COGNITIVE SEARCH ADMIN API KEY - e.g. cognitive-search-admin-api-key>"
```

## Setup Azure OpenAI

We'll start as usual by defining our Azure OpenAI service API key and endpoint details, specifying the model deployment we want to use and then we'll initiate a connection to the Azure OpenAI service.

**NOTE**: As with previous labs, we'll use the values from the `.env` file in the root of this repository.

In [ ]:
// Add the Packages
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.7"
#r "nuget: Azure.Identity, 1.10.1"
#r "nuget: Azure.Search.Documents, 11.5.0-beta.4"

using System.IO;

// Read values from .env file
DotNetEnv.Env.Load("../../../.env");

// Load values into variables
var openai_api_type = System.Environment.GetEnvironmentVariable("OPENAI_API_TYPE");
var openai_api_key = System.Environment.GetEnvironmentVariable("OPENAI_API_KEY");
var openai_api_base = System.Environment.GetEnvironmentVariable("OPENAI_API_BASE");
var openai_api_version = System.Environment.GetEnvironmentVariable("OPENAI_API_VERSION");
var deployment_name = System.Environment.GetEnvironmentVariable("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME");
var embedding_name = System.Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME");
var acs_service_name = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_SERVICE_NAME");
var acs_endpoint_name = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_ENDPOINT_NAME");
var acs_index_name = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_INDEX_NAME");
var acs_api_key = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_API_KEY");

Console.WriteLine("Environment Variables loaded.");

First, we will load the data from the movies.csv file and then extract a subset to load into the Azure Cognitive Search index. We do this to help avoid the Azure OpenAI embedding limits and long loading times when inserting data into the index.

In [ ]:
using System.IO;

// Movie Fields in CSV
// id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
string path = @"./movies.csv";
string[] allMovies;
string[] movieSubset;
// The subset of movies to load into the Azure Cognitive Search Index.
// The more movies you load, the longer it will take due to Embedding limits.
int movieSubsetCount = 50;
try
{
    if (File.Exists(path))
    {
        allMovies = File.ReadAllLines(path);
        movieSubset = allMovies.Skip(1).Take(movieSubsetCount).ToArray();
    }
}
catch (Exception e)
{
    Console.WriteLine("The process failed: {0}", e.ToString());
}

// Write out the results.
Console.WriteLine($"CSV File Loaded {movieSubset.Length}.");

Next, we will create an Azure OpenAI Client to do embeddings and completions.

In [ ]:
using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;

// Create the Azure OpenAI client.
OpenAIClient azureOpenAIClient = new OpenAIClient(new Uri(openai_api_base),new AzureKeyCredential(openai_api_key));

Console.WriteLine("Azure OpenAI client Created.");

## Load Movies into Azure Cognitive Search

Next, we'll create the Azure Cognitive Search index, embed the loaded movies from the CSV file, and upload the data into the newly created index. Depending on the number of movies loaded and rate limiting, this might take a while to do the embeddings so be patient.

In [ ]:
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;
using Microsoft.Extensions.Configuration;

// Setup variables.
string vectorSearchConfigName = "my-vector-config";
string semanticSearchConfigName = "my-semantic-config";
int modelDimensions = 1536;

// Create an Azure Cognitive Search index client.
AzureKeyCredential indexCredential = new AzureKeyCredential(acs_api_key);
SearchIndexClient indexClient = new SearchIndexClient(new Uri(acs_endpoint_name), indexCredential);

// Create the index definition.
// Setup the VectorSearch configuration.
VectorSearch vectorSearch = new VectorSearch();
vectorSearch.AlgorithmConfigurations.Add(new HnswVectorSearchAlgorithmConfiguration(vectorSearchConfigName));
// Setup the SemanticSettings configuration.
SemanticSettings semanticSettings = new SemanticSettings();
semanticSettings.Configurations.Add(new SemanticConfiguration(semanticSearchConfigName, new PrioritizedFields()
{
    TitleField = new SemanticField() { FieldName = "content" },
    KeywordFields = { new SemanticField() { FieldName = "content" } },
    ContentFields = { new SemanticField() { FieldName = "content" } },
}));
// Setup the Fields configuration.
IList<SearchField> fields = new List<SearchField>();
fields.Add(new SimpleField("id", SearchFieldDataType.String) { IsKey = true, IsFilterable = true, IsSortable = true, IsFacetable = true });
fields.Add(new SearchableField("content", false) {});
fields.Add(new SearchField("content_vector", SearchFieldDataType.Collection(SearchFieldDataType.Single))
{
    IsSearchable = true,
    VectorSearchDimensions = modelDimensions,
    VectorSearchConfiguration = vectorSearchConfigName
});

// Setup SearchIndex
SearchIndex searchIndex = new SearchIndex(acs_index_name);
searchIndex.VectorSearch = vectorSearch;
searchIndex.SemanticSettings = semanticSettings;
searchIndex.Fields = fields;

// Create the index
indexClient.CreateOrUpdateIndex(searchIndex);

Console.WriteLine($"Index {acs_index_name} created.");

Next we will create the document structure needed to upload the data into the Azure Cognitive Search index.

**NOTE**: Be patient, creating the embeddings will take some time due to the Azure OpenAI embedding Token Per Minute (TPM) limits.

In [ ]:
using Azure.Search.Documents;
using Azure.Search.Documents.Models;

// Create structure to match the Azure Cognitive Search index.
List<SearchDocument> movieDocuments = new List<SearchDocument>();
for (int i = 0; i < movieSubset.Length; i++) 
{
    Console.WriteLine($"Movie {i} being added.");
    string id = System.Guid.NewGuid().ToString();
    string content = movieSubset[i];
    float[] contentEmbeddings = azureOpenAIClient.GetEmbeddings(embedding_name, new EmbeddingsOptions(content)).Value.Data[0].Embedding.ToArray();
    SearchDocument document = new SearchDocument();
    document.Add("id", id);
    document.Add("content", content);
    document.Add("content_vector", contentEmbeddings);
    movieDocuments.Add(new SearchDocument(document));
    Console.WriteLine($"Movie {i} added.");
}

Console.WriteLine($"New SearchDocument structure with embeddings created for {movieDocuments.Count} movies.");

Next we will upload the movie documents in the newly created structure to the Azure Cognitive Search index.

In [ ]:
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;

// Create an Azure Cognitive Search index client.
AzureKeyCredential indexCredential = new AzureKeyCredential(acs_api_key);
SearchIndexClient indexClient = new SearchIndexClient(new Uri(acs_endpoint_name), indexCredential);
SearchClient searchIndexClient = indexClient.GetSearchClient(acs_index_name);

IndexDocumentsOptions options = new IndexDocumentsOptions { ThrowOnAnyError = true };
searchIndexClient.IndexDocuments(IndexDocumentsBatch.Upload(movieDocuments), options);

Console.WriteLine($"Successfully loaded {movieDocuments.Count} movies into Azure Cognitive Search index.");

## Vector Store Searching using Azure Cognitive Search

Now that we have the movies loaded into Azure Cognitive Search, let's do some different types of searches using the Azure Cognitive Search SDK.

In [ ]:
// Setup an Azure Cognitive Search client for searching.

using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using System.Text.Json.Serialization;

// Create a client
AzureKeyCredential credential = new AzureKeyCredential(acs_api_key);
SearchClient searchClient = new SearchClient(new Uri(acs_endpoint_name), acs_index_name, credential);

Console.WriteLine($"Successfully created Azure Cognitive Search SearchClient.");

In [ ]:
// First, let's do a plain vanilla text search, no vectors or embeddings.

using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using System.Text.Json.Serialization;

var query = "What are the best 80s movies I should look at?";

SearchOptions searchOptions = new SearchOptions
{
    // Filter to only Content greater than or equal our preference
    // Filter = SearchFilter.Create($"Content ge {content}"),
    // OrderBy = { "Content desc" } // Sort by Content from high to low
    // Size = 5, // Take only 5 results
    // Select = { "id", "content", "content_vector" }, // Which fields to return
    Size = 5,
    Select = { "content" },
};

SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(query, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

// Print count of total results.
Console.WriteLine($"Returned {results.Count()} results using only text-based search.");
Console.WriteLine("----------");

// Iterate over Results
// Index Fields - id, content, content_vector
foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Movie: {result.Document["content"]}");
    Console.WriteLine("----------");
};

In [ ]:
// Now let's do a vector search that uses the embeddings we created and inserted into content_vector field in the index.

using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using System.Text.Json.Serialization;

var query = "What are the best 80s movies I should look at?";

float[] queryEmbedding = azureOpenAIClient.GetEmbeddings(embedding_name, new EmbeddingsOptions(query)).Value.Data[0].Embedding.ToArray();

// Note the Vectors addition in the SearchOptions
SearchOptions searchOptions = new SearchOptions
{
    // Filter to only Content greater than or equal our preference
    // Filter = SearchFilter.Create($"Content ge {content}"),
    // OrderBy = { "Content desc" } // Sort by Content from high to low
    // Size = 5, // Take only 5 results
    // Select = { "id", "content", "content_vector" }, // Which fields to return
    Vectors = { new() { Value = queryEmbedding, KNearestNeighborsCount = 5, Fields = { "content_vector" } } }, // Vector Search
    Size = 5,
    Select = { "content" },
};

// Note the search text is null when doing a vector search.
SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(null, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

// Print count of total results.
Console.WriteLine($"Returned {results.Count()} results using only vector-based search.");
Console.WriteLine("----------");

// Iterate over Results
// Index Fields - id, content, content_vector
foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Movie: {result.Document["content"]}");
    Console.WriteLine("----------");
};

Did that return what you expected? Probably not, let's dig deeper to see why.

Let's do the same search again, but this time let's return the **Search Score** so we can see the value returned by the cosine similarity vector store calculation.

In [ ]:
// Try again, but this time let's add the relevance score to maybe see why

using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using System.Text.Json.Serialization;

var query = "What are the best 80s movies I should look at?";

float[] queryEmbedding = azureOpenAIClient.GetEmbeddings(embedding_name, new EmbeddingsOptions(query)).Value.Data[0].Embedding.ToArray();

// Note the Vectors addition in the SearchOptions
SearchOptions searchOptions = new SearchOptions
{
    // Filter to only Content greater than or equal our preference
    // Filter = SearchFilter.Create($"Content ge {content}"),
    // OrderBy = { "Content desc" } // Sort by Content from high to low
    // Size = 5, // Take only 5 results
    // Select = { "id", "content", "content_vector" }, // Which fields to return
    Vectors = { new() { Value = queryEmbedding, KNearestNeighborsCount = 5, Fields = { "content_vector" } } }, // Vector Search
    Size = 5,
    Select = { "id", "content" },
};

// Note the search text is null when doing a vector search.
SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(null, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

// Print count of total results.
Console.WriteLine($"Returned {results.Count()} results using only vector-based search.");
Console.WriteLine("----------");

// Iterate over Results
// Index Fields - id, content, content_vector
foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Id: {result.Document["id"]}");
    Console.WriteLine($"Score: {result.Score}");
    Console.WriteLine("----------");
};

If you look at the Search Score you will see the relevant ranking of the closest vector match to the query inputted. The lower the score the farther apart the two vectors are. Let's change the search term and see if we can get a higher Search Score which means a higher match and closer vector proximity.

In [ ]:
// Try again, but this time let's add the relevance score to maybe see why

using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using System.Text.Json.Serialization;

var query = "Who are the actors in the movie Hidden Figures?";

float[] queryEmbedding = azureOpenAIClient.GetEmbeddings(embedding_name, new EmbeddingsOptions(query)).Value.Data[0].Embedding.ToArray();

// Note the Vectors addition in the SearchOptions
SearchOptions searchOptions = new SearchOptions
{
    // Filter to only Content greater than or equal our preference
    // Filter = SearchFilter.Create($"Content ge {content}"),
    // OrderBy = { "Content desc" } // Sort by Content from high to low
    // Size = 5, // Take only 5 results
    // Select = { "id", "content", "content_vector" }, // Which fields to return
    Vectors = { new() { Value = queryEmbedding, KNearestNeighborsCount = 5, Fields = { "content_vector" } } }, // Vector Search
    Size = 5,
    Select = { "id", "content" },
};

// Note the search text is null when doing a vector search.
SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(null, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

// Print count of total results.
Console.WriteLine($"Returned {results.Count()} results using only vector-based search.");
Console.WriteLine("----------");

// Iterate over Results
// Index Fields - id, content, content_vector
foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Id: {result.Document["id"]}");
    Console.WriteLine($"Score: {result.Score}");
    Console.WriteLine("----------");
};

**NOTE:** As you have seen from the results, different inputs can return different results, it all depends on what data is in the Vector Store. The higher the score the higher the likelihood of a match.

## Hybrid Searching using Azure Cognitive Search

What is Hybrid Search? The search is implemented at the field level, which means you can build queries that include vector fields and searchable text fields. The queries execute in parallel and the results are merged into a single response. Optionally, add semantic search, currently in preview, for even more accuracy with L2 reranking using the same language models that power Bing.

**NOTE:** Hybrid Search is a key value proposition of Azure Cognitive Search in comparison to vector only data stores.

In [ ]:
// Hybrid Search
// Let's try our original query again using Hybrid Search (ie. Combination of Text & Vector Search)

using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using System.Text.Json.Serialization;

var query = "What are the best 80s movies I should look at?";

float[] queryEmbedding = azureOpenAIClient.GetEmbeddings(embedding_name, new EmbeddingsOptions(query)).Value.Data[0].Embedding.ToArray();

// Note the Vectors addition in the SearchOptions
SearchOptions searchOptions = new SearchOptions
{
    // Filter to only Content greater than or equal our preference
    // Filter = SearchFilter.Create($"Content ge {content}"),
    // OrderBy = { "Content desc" } // Sort by Content from high to low
    // Size = 5, // Take only 5 results
    // Select = { "id", "content", "content_vector" }, // Which fields to return
    Vectors = { new() { Value = queryEmbedding, KNearestNeighborsCount = 5, Fields = { "content_vector" } } }, // Vector Search
    Size = 5,
    Select = { "id", "content" },
};

// Note the search text and the vector search are both filled in.
SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(query, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

// Print count of total results.
Console.WriteLine($"Returned {results.Count()} results using hybrid search search.");
Console.WriteLine("----------");

// Iterate over Results
// Index Fields - id, content, content_vector
foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Id: {result.Document["id"]}");
    Console.WriteLine($"Score: {result.Score}");
    Console.WriteLine("----------");
};

In [ ]:
// Hybrid Search
// Let's try our more specific query again to see the difference in the score returned.

using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using System.Text.Json.Serialization;

var query = "Who are the actors in the movie Hidden Figures?";

float[] queryEmbedding = azureOpenAIClient.GetEmbeddings(embedding_name, new EmbeddingsOptions(query)).Value.Data[0].Embedding.ToArray();

// Note the Vectors addition in the SearchOptions
SearchOptions searchOptions = new SearchOptions
{
    // Filter to only Content greater than or equal our preference
    // Filter = SearchFilter.Create($"Content ge {content}"),
    // OrderBy = { "Content desc" } // Sort by Content from high to low
    // Size = 5, // Take only 5 results
    // Select = { "id", "content", "content_vector" }, // Which fields to return
    Vectors = { new() { Value = queryEmbedding, KNearestNeighborsCount = 5, Fields = { "content_vector" } } }, // Vector Search
    Size = 5,
    Select = { "id", "content" },
};

// Note the search text and the vector search are both filled in.
SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(query, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();

// Print count of total results.
Console.WriteLine($"Returned {results.Count()} results using hybrid search search.");
Console.WriteLine("----------");

// Iterate over Results
// Index Fields - id, content, content_vector
foreach (SearchResult<SearchDocument> result in results)
{
    Console.WriteLine($"Id: {result.Document["id"]}");
    Console.WriteLine($"Score: {result.Score}");
    Console.WriteLine("----------");
};

## Bringing it All Together with Retrieval Augmented Generation (RAG) + Semantic Kernel (SK)

Now that we have our Vector Store setup and data loaded, we are now ready to implement the RAG pattern using AI Orchestration. At a high-level, the following steps are required:
1. Ask the question
2. Create Prompt Template with inputs
3. Get Embedding representation of inputted question
4. Use embedded version of the question to search Azure Cognitive Search (ie. The Vector Store)
5. Inject the results of the search into the Prompt Template & Execute the Prompt to get the completion

In [ ]:
// Implement RAG using Semantic Kernel (SK)

// Add the Packages
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 0.24.230918.1-preview"
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.7"
#r "nuget: Azure.Identity, 1.10.1"
#r "nuget: Azure.Search.Documents, 11.5.0-beta.4"
#r "nuget: Microsoft.Extensions.Logging, 7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 7.0.0"

using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.SkillDefinition;
using Azure;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using Azure.Search.Documents.Indexes;
using Azure.AI.OpenAI;
using Azure.Identity;

// Read values from .env file
DotNetEnv.Env.Load("../../../.env");

// Load values into variables
var openai_api_type = System.Environment.GetEnvironmentVariable("OPENAI_API_TYPE");
var openai_api_key = System.Environment.GetEnvironmentVariable("OPENAI_API_KEY");
var openai_api_base = System.Environment.GetEnvironmentVariable("OPENAI_API_BASE");
var openai_api_version = System.Environment.GetEnvironmentVariable("OPENAI_API_VERSION");
var deployment_name = System.Environment.GetEnvironmentVariable("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME");
var embedding_name = System.Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME");
var acs_service_name = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_SERVICE_NAME");
var acs_endpoint_name = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_ENDPOINT_NAME");
var acs_index_name = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_INDEX_NAME");
var acs_api_key = System.Environment.GetEnvironmentVariable("AZURE_COGNITIVE_SEARCH_API_KEY");

Console.WriteLine("Environment Variables loaded.");

// Setup Semantic Kernel
IKernel kernel = Kernel.Builder
    .WithAzureChatCompletionService(deployment_name, openai_api_base, openai_api_key)
    .WithAzureTextEmbeddingGenerationService(embedding_name, openai_api_base, openai_api_key)
    .Build();

Console.WriteLine("SK Kernel with ChatCompletion and EmbeddingsGeneration services created.");

// Ask the question
var question = "List the movies about ships on the water.";

// Create a prompt template with variables, note the double curly braces with dollar sign for the variables
// First let's create the prompt string.
var sk_prompt = @"
Question: {{$original_question}}

Do not use any other data.
Only use the movie data below when responding.
{{$search_results}}
";
// Create the PromptTemplateConfig
PromptTemplateConfig promptConfig = new PromptTemplateConfig
{
    Schema = 1,
    Type = "completion",
    Description = "Gets the intent of the user.",
    Completion = 
    {
        Temperature = 0.1,
        TopP = 0.5,
        PresencePenalty = 0.0,
        FrequencyPenalty = 0.0,
        MaxTokens = 500
        // StopSequences = null,
        // ChatSystemPprompt = null;
    },
    Input = 
    {
        Parameters = new List<PromptTemplateConfig.InputParameter>
        {
            new PromptTemplateConfig.InputParameter
            {
                Name="original_question",
                Description="The user's request.",
                DefaultValue=""
            },
            new PromptTemplateConfig.InputParameter
            {
                Name="search_results",
                Description="Vector Search results from Azure Cognitive Search.",
                DefaultValue=""
            }
        }
    }
};
// Create the SemanticFunctionConfig object
PromptTemplate promptTemplate = new PromptTemplate(
    sk_prompt,
    promptConfig,
    kernel
);
SemanticFunctionConfig functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);
// Register the GetIntent function with the Kernel
ISKFunction getIntentFunction = kernel.RegisterSemanticFunction("CustomPlugin", "GetIntent", functionConfig);

Console.WriteLine("Semantic Function GetIntent with SK has been completed.");

// Get Embedding for the original question
OpenAIClient azureOpenAIClient = new OpenAIClient(new Uri(openai_api_base),new AzureKeyCredential(openai_api_key));
float[] questionEmbedding = azureOpenAIClient.GetEmbeddings(embedding_name, new EmbeddingsOptions(question)).Value.Data[0].Embedding.ToArray();

Console.WriteLine("Embedding of original question has been completed.");

// Search Vector Store
SearchOptions searchOptions = new SearchOptions
{
    // Filter to only Content greater than or equal our preference
    // Filter = SearchFilter.Create($"Content ge {content}"),
    // OrderBy = { "Content desc" } // Sort by Content from high to low
    // Size = 5, // Take only 5 results
    // Select = { "id", "content", "content_vector" }, // Which fields to return
    Vectors = { new() { Value = questionEmbedding, KNearestNeighborsCount = 5, Fields = { "content_vector" } } }, // Vector Search
    Size = 5,
    Select = { "id", "content" },
};

// Note the search text is null and the vector search is filled in.
AzureKeyCredential credential = new AzureKeyCredential(acs_api_key);
SearchClient searchClient = new SearchClient(new Uri(acs_endpoint_name), acs_index_name, credential);
SearchResults<SearchDocument> response = searchClient.Search<SearchDocument>(null, searchOptions);
Pageable<SearchResult<SearchDocument>> results = response.GetResults();
// Create string from the results
StringBuilder stringBuilderResults = new StringBuilder();
foreach (SearchResult<SearchDocument> result in results)
{
    stringBuilderResults.AppendLine($"{result.Document["content"]}");
};

Console.WriteLine("Searching of Vector Store has been completed.");

// Build the Prompt and Execute against the Azure OpenAI to get the completion
// Initialize the prompt variables
ContextVariables variables = new ContextVariables
{
    ["original_question"] = question,
    ["search_results"] = stringBuilderResults.ToString()
};
// Use SK Chaining to Invoke Semantic Function
string completion = (await kernel.RunAsync(variables, getIntentFunction)).Result;
Console.WriteLine(completion);

Console.WriteLine("Implementation of RAG using SK, C# and Azure Cognitive Search has been completed.");

## Next Section

📣 [Deploy AI](../../04-deploy-ai/README.md)